# 1. 결정 트리 훈련

1-a . make_moons (n_sample=1000, noise=0.4)를 사용해 데이터셋을 생성합니다.

In [49]:
from sklearn.datasets import make_moons

X,y = make_moons(n_samples=1000, noise=0.4, random_state=42)

1-b. 이를 train_test_split( )을 사용해 훈련 세트와 테스트 세트로 나눕니다.(random_state=42, train:test = 8:2 비율로)


In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

1-c. DecisionTreeClassifier의 최적의 매개변수를 찾기 위해 교차 검증과 함께 그리드 탐색을 수행합니다. (GridSearchCV)

힌트 : 여러가지 max_leaf_nodes 값을 시도해보세요! (교재 pg. 113, 207 참고)

In [71]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

# 예시 파라미터: params = {'max_leaf_nodes' : list(range(2,100)),'min_samples_split' : [2,3,4]}

params = {'max_leaf_nodes' : list(range(2,100)), 'min_samples_split':[2,3,4]}

grid_search_cv = GridSearchCV(estimator = DecisionTreeClassifier(random_state=42), param_grid=params, cv=5, scoring='accuracy')
grid_search_cv.fit(X_train, y_train)

print("최적 파라미터:", grid_search_cv.best_params_)

최적 파라미터: {'max_leaf_nodes': 6, 'min_samples_split': 2}


In [73]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

# 예시 파라미터: params = {'max_leaf_nodes' : list(range(2,100)),'min_samples_split' : [2,3,4]}

params2 = {'max_leaf_nodes' : list(range(2,100))}

grid_search_cv2 = GridSearchCV(estimator = DecisionTreeClassifier(random_state=42), param_grid=params2, cv=5, scoring='accuracy')
grid_search_cv2.fit(X_train, y_train)

print("최적 파라미터:", grid_search_cv2.best_params_)

최적 파라미터: {'max_leaf_nodes': 6}


In [63]:
cv_results_df = pd.DataFrame(grid_search_cv.cv_results_)
sorted_results = cv_results_df[
    ['param_max_leaf_nodes', 'param_min_samples_split', 'mean_test_score']
].sort_values(by='mean_test_score', ascending=False)

print(sorted_results)

    param_max_leaf_nodes param_min_samples_split  mean_test_score
14                     6                       4          0.85000
12                     6                       2          0.85000
13                     6                       3          0.85000
15                     7                       2          0.84875
17                     7                       4          0.84875
..                   ...                     ...              ...
292                   99                       3          0.79250
291                   99                       2          0.79125
1                      2                       3          0.76500
2                      2                       4          0.76500
0                      2                       2          0.76500

[294 rows x 3 columns]


In [77]:
cv_results_df2 = pd.DataFrame(grid_search_cv2.cv_results_)
sorted_results2 = cv_results_df2[
    ['param_max_leaf_nodes', 'mean_test_score']
].sort_values(by='mean_test_score', ascending=False)

print(sorted_results2)

   param_max_leaf_nodes  mean_test_score
4                     6          0.85000
5                     7          0.84875
2                     4          0.84875
3                     5          0.84875
6                     8          0.84750
..                  ...              ...
93                   95          0.79875
96                   98          0.79750
95                   97          0.79750
97                   99          0.79125
0                     2          0.76500

[98 rows x 2 columns]


In [22]:
# 최적의 매개변수 확인
grid_search_cv.best_estimator_

DecisionTreeClassifier(max_leaf_nodes=6, random_state=42)

1-d. 찾은 매개변수를 사용해 전체 훈련 세트에 대해 모델을 훈련시키고 테스트 세트에서 성능(정확도)을 측정합니다. (대략 85~87% 정도 나옵니다.)

In [24]:
# GridSearchCV는 최적의 모델로 다시 훈련시키기 때문에 별도의 훈련이 필요없습니다.
from sklearn.metrics import accuracy_score
best_dt = grid_search_cv.best_estimator_
y_pred = best_dt.predict(X_test)
test_acc = accuracy_score(y_test, y_pred)
print(f"Test set accuracy: {test_acc:.4f}")

Test set accuracy: 0.8550


In [79]:
from sklearn.metrics import accuracy_score
best_dt = grid_search_cv.best_estimator_
y_pred = best_dt.predict(X_test)
test_acc = accuracy_score(y_test, y_pred)
print(f"Test set accuracy: {test_acc:.4f}")

Test set accuracy: 0.8550


# 2. 랜덤포레스트

In [9]:
# 데이터 다운로드 링크로 데이터를 코랩에 불러옵니다.

!wget 'https://bit.ly/3i4n1QB'

import zipfile
with zipfile.ZipFile('3i4n1QB', 'r') as existing_zip:
    existing_zip.extractall('data')

'wget'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


FileNotFoundError: [Errno 2] No such file or directory: '3i4n1QB'

In [15]:
import os
import urllib.request
import zipfile

# 1) 다운로드할 URL과 저장할 파일명 지정
url = 'https://bit.ly/3i4n1QB'
local_zip = 'data.zip'

# 2) URL에서 파일 다운로드
urllib.request.urlretrieve(url, local_zip)
print(f"Downloaded to {local_zip}")

# 3) 압축 풀 폴더 만들기
extract_dir = 'data'
os.makedirs(extract_dir, exist_ok=True)

# 4) ZIP 파일 압축 해제
with zipfile.ZipFile(local_zip, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)
print(f"Extracted all files to ./{extract_dir}/")

Downloaded to data.zip
Extracted all files to ./data/


In [26]:
# 라이브러리 불러오기

import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

In [30]:
train = pd.read_csv('data/train.csv')

# Scaling
scaler = MinMaxScaler()
scaler.fit(train[['fixed acidity']])
train['Scaled fixed acidity'] = scaler.transform(train[['fixed acidity']])

# Encoding
encoder = OneHotEncoder()
encoder.fit(train[['type']])
onehot = encoder.transform(train[['type']])
onehot = onehot.toarray()
onehot = pd.DataFrame(onehot)
onehot.columns = encoder.get_feature_names_out()
train = pd.concat([train, onehot], axis = 1)
train = train.drop(columns = ['type'])
train.head()

,index,quality,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,Scaled fixed acidity,type_red,type_white
0,0,5,5.6,0.695,0.06,6.8,0.042,9.0,84.0,0.99432,3.44,0.44,10.2,0.148760,0.0,1.0
1,1,5,8.8,0.610,0.14,2.4,0.067,10.0,42.0,0.99690,3.19,0.59,9.5,0.413223,1.0,0.0
2,2,5,7.9,0.210,0.39,2.0,0.057,21.0,138.0,0.99176,3.05,0.52,10.9,0.338843,0.0,1.0
3,3,6,7.0,0.210,0.31,6.0,0.046,29.0,108.0,0.99390,3.26,0.50,10.8,0.264463,0.0,1.0
4,4,6,7.8,0.400,0.26,9.5,0.059,32.0,178.0,0.99550,3.04,0.43,10.9,0.330579,0.0,1.0


2-a. 랜덤포레스트 분류 모형을 불러옵니다.

In [32]:
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(random_state=42)

2-b. 랜덤포레스트 분류 모형을 "random_classifier"라는 변수에 저장합니다.

In [36]:
random_classifier = RandomForestClassifier(random_state=42)

2-c. "X"라는 변수에 train의 "quality" 피쳐를 제거하고 저장합니다.

In [38]:
X = train.drop(columns=['quality'])

2-d. "y"라는 변수에 정답인 train의 "quality" 피쳐를 저장합니다.

In [40]:
y = train['quality']

2-e. "random_classifier"를 X와 y를 이용해 학습시켜보세요.

In [42]:
random_classifier.fit(X, y)

train_acc = random_classifier.score(X, y)
print(f"Training set accuracy: {train_acc:.4f}")

Training set accuracy: 1.0000
